# Gráficos de Cleveland: Diferencias en Tiempo entre Generales y Ballotage

Este notebook genera **gráficos de Cleveland (Cleveland dot plots)** para visualizar las diferencias en Cambio de Tiempo (CT) entre candidatos de Izquierda y Derecha, comparando Generales vs Ballotage.

## Concepto:

Para cada ítem mostramos:
- **Punto azul (●)**: Diferencia Izq-Der en **Generales** (Dif_Gen_CT)
- **Punto rojo (●)**: Diferencia Izq-Der en **Ballotage** (Dif_Bal_CT)
- **Línea conectando**: Muestra el cambio entre elecciones
- **Color de línea**:
  - Verde: Diferencia aumentó en Ballotage (más positivo)
  - Gris: Cambio pequeño (≤ 0.5 segundos)
  - Naranja: Diferencia disminuyó en Ballotage (más negativo)

## Interpretación:

- **Valores positivos**: Mayor tiempo en candidatos de Izquierda
- **Valores negativos**: Mayor tiempo en candidatos de Derecha
- **Línea hacia la derecha**: La diferencia aumentó en Ballotage
- **Línea hacia la izquierda**: La diferencia disminuyó en Ballotage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Ruta al archivo
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Archivo_Elecciones = os.path.join(Ruta_Base, 'df_Elecciones.xlsx')

# Cargar datos
df_Elecciones = pd.read_excel(Archivo_Elecciones)

print(f'✓ Datos cargados:')
print(f'  - {len(df_Elecciones)} registros')
print(f'  - {len(df_Elecciones.columns)} columnas')

# Verificar que tenemos las columnas necesarias
columnas_dif_gen = [col for col in df_Elecciones.columns if col.startswith('Dif_Gen_CT')]
columnas_dif_bal = [col for col in df_Elecciones.columns if col.startswith('Dif_Bal_CT')]

print(f'\n  - Variables Dif_Gen_CT: {len(columnas_dif_gen)}')
print(f'  - Variables Dif_Bal_CT: {len(columnas_dif_bal)}')

## 2. Preparar Datos para Gráfico

In [ ]:
# Definir ítems
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
Todos_Items = sorted(Items_Progresistas + Items_Conservadores)

print(f'Total de ítems: {len(Todos_Items)}')
print(f'Ítems: {Todos_Items}')

# Calcular promedios por ítem
datos_grafico = []

for item in Todos_Items:
    var_gen = f'Dif_Gen_CT_Item_{item}'
    var_bal = f'Dif_Bal_CT_Item_{item}'
    
    if var_gen in df_Elecciones.columns and var_bal in df_Elecciones.columns:
        # Calcular promedios (ignorando NaN)
        media_gen = df_Elecciones[var_gen].mean()
        media_bal = df_Elecciones[var_bal].mean()
        
        # Calcular cambio
        cambio = media_bal - media_gen
        
        # Determinar tipo de ítem
        tipo = 'Progresista' if item in Items_Progresistas else 'Conservador'
        
        datos_grafico.append({
            'Item': item,
            'Tipo': tipo,
            'Dif_Generales': media_gen,
            'Dif_Ballotage': media_bal,
            'Cambio': cambio
        })

# Crear DataFrame
df_grafico = pd.DataFrame(datos_grafico)

print(f'\n✓ Datos preparados para {len(df_grafico)} ítems')
print(f'\nPrimeras filas:')
print(df_grafico.head(10))

## 3. Función para Crear Gráfico de Cleveland

In [ ]:
def Crear_Grafico_Cleveland(df, titulo, nombre_archivo=None, carpeta_destino='Graficos_Cleveland'):
    """
    Crea un gráfico de Cleveland mostrando diferencias Gen vs Bal.
    
    Parámetros:
    -----------
    df : DataFrame
        DataFrame con columnas: Item, Dif_Generales, Dif_Ballotage, Cambio
    titulo : str
        Título del gráfico
    nombre_archivo : str, optional
        Nombre del archivo (sin extensión)
    carpeta_destino : str
        Carpeta donde guardar
    """
    
    # Crear carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Ordenar por cambio (descendente)
    df_sorted = df.sort_values('Cambio', ascending=True).reset_index(drop=True)
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Parámetros visuales
    y_positions = np.arange(len(df_sorted))
    
    # Dibujar líneas conectando los puntos
    for idx, row in df_sorted.iterrows():
        gen_val = row['Dif_Generales']
        bal_val = row['Dif_Ballotage']
        cambio = row['Cambio']
        
        # Determinar color de línea según cambio
        if cambio > 0.5:  # Aumentó
            color_linea = '#2ecc71'  # Verde
            alpha = 0.6
        elif cambio < -0.5:  # Disminuyó
            color_linea = '#e74c3c'  # Rojo/Naranja
            alpha = 0.6
        else:  # Cambio pequeño
            color_linea = '#95a5a6'  # Gris
            alpha = 0.3
        
        # Dibujar línea
        ax.plot([gen_val, bal_val], [idx, idx], 
                color=color_linea, linewidth=1.5, alpha=alpha, zorder=1)
    
    # Dibujar puntos de Generales
    ax.scatter(df_sorted['Dif_Generales'], y_positions, 
               s=120, c='#3498db', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Generales', zorder=3, alpha=0.8)
    
    # Dibujar puntos de Ballotage
    ax.scatter(df_sorted['Dif_Ballotage'], y_positions, 
               s=120, c='#e74c3c', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Ballotage', zorder=3, alpha=0.8)
    
    # Línea vertical en x=0
    ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3, zorder=0)
    
    # Configurar ejes
    ax.set_yticks(y_positions)
    ax.set_yticklabels([f'Ítem {int(item)}' for item in df_sorted['Item']], fontsize=10)
    
    ax.set_xlabel('Diferencia Izq - Der (segundos)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Ítem', fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    
    # Grid
    ax.grid(True, axis='x', alpha=0.3, linestyle=':', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Leyenda personalizada
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', 
               markersize=10, label='Generales', markeredgecolor='white', markeredgewidth=1),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#e74c3c', 
               markersize=10, label='Ballotage', markeredgecolor='white', markeredgewidth=1),
        Line2D([0], [0], color='#2ecc71', linewidth=2, label='Aumentó (>0.5s)'),
        Line2D([0], [0], color='#95a5a6', linewidth=2, label='Sin cambio (±0.5s)'),
        Line2D([0], [0], color='#e74c3c', linewidth=2, label='Disminuyó (<-0.5s)')
    ]
    
    ax.legend(handles=legend_elements, loc='lower right', fontsize=10, 
              framealpha=0.95, edgecolor='gray')
    
    # Ajustar layout
    plt.tight_layout()
    
    # Guardar
    if nombre_archivo:
        ruta_completa = os.path.join(carpeta_destino, f'{nombre_archivo}.png')
        plt.savefig(ruta_completa, dpi=300, bbox_inches='tight', facecolor='white')
        print(f'✅ Gráfico guardado: {ruta_completa}')
    
    plt.show()
    
    return fig, ax

## 4. Gráfico Principal: Todos los Ítems

In [ ]:
# Crear gráfico principal
fig, ax = Crear_Grafico_Cleveland(
    df_grafico,
    titulo='Diferencias en Tiempo (Izq - Der):\nGenerales vs Ballotage',
    nombre_archivo='Cleveland_CT_Todos_Items'
)

## 5. Análisis Estadístico de Cambios

In [ ]:
print('='*70)
print('ANÁLISIS DE CAMBIOS ENTRE GENERALES Y BALLOTAGE')
print('='*70)

print(f'\n📊 Estadísticas Generales:')
print(f'  Promedio Dif_Generales: {df_grafico["Dif_Generales"].mean():.3f} seg')
print(f'  Promedio Dif_Ballotage: {df_grafico["Dif_Ballotage"].mean():.3f} seg')
print(f'  Promedio Cambio: {df_grafico["Cambio"].mean():.3f} seg')

print(f'\n📈 Distribución de Cambios:')
aumentaron = len(df_grafico[df_grafico['Cambio'] > 0.5])
sin_cambio = len(df_grafico[(df_grafico['Cambio'] >= -0.5) & (df_grafico['Cambio'] <= 0.5)])
disminuyeron = len(df_grafico[df_grafico['Cambio'] < -0.5])

print(f'  Aumentaron (>0.5s): {aumentaron} ítems ({aumentaron/len(df_grafico)*100:.1f}%)')
print(f'  Sin cambio (±0.5s): {sin_cambio} ítems ({sin_cambio/len(df_grafico)*100:.1f}%)')
print(f'  Disminuyeron (<-0.5s): {disminuyeron} ítems ({disminuyeron/len(df_grafico)*100:.1f}%)')

print(f'\n🏆 Top 5 Ítems con MAYOR Aumento:')
print('-'*70)
top_aumento = df_grafico.nlargest(5, 'Cambio')[['Item', 'Tipo', 'Dif_Generales', 'Dif_Ballotage', 'Cambio']]
print(top_aumento.to_string(index=False))

print(f'\n🏆 Top 5 Ítems con MAYOR Disminución:')
print('-'*70)
top_disminucion = df_grafico.nsmallest(5, 'Cambio')[['Item', 'Tipo', 'Dif_Generales', 'Dif_Ballotage', 'Cambio']]
print(top_disminucion.to_string(index=False))

print('\n' + '='*70)

## 6. Comparación por Tipo de Ítem

In [ ]:
print('='*70)
print('COMPARACIÓN: ÍTEMS PROGRESISTAS vs CONSERVADORES')
print('='*70)

# Agrupar por tipo
por_tipo = df_grafico.groupby('Tipo').agg({
    'Dif_Generales': ['mean', 'std'],
    'Dif_Ballotage': ['mean', 'std'],
    'Cambio': ['mean', 'std'],
    'Item': 'count'
}).round(3)

print('\n📊 Estadísticas por Tipo:')
print(por_tipo)

print('\n💡 Interpretación:')
cambio_prog = df_grafico[df_grafico['Tipo'] == 'Progresista']['Cambio'].mean()
cambio_cons = df_grafico[df_grafico['Tipo'] == 'Conservador']['Cambio'].mean()

print(f'\n  Ítems Progresistas:')
print(f'    - Cambio promedio: {cambio_prog:.3f} seg')
if cambio_prog > 0:
    print(f'    - En Ballotage aumentó la diferencia Izq-Der')
else:
    print(f'    - En Ballotage disminuyó la diferencia Izq-Der')

print(f'\n  Ítems Conservadores:')
print(f'    - Cambio promedio: {cambio_cons:.3f} seg')
if cambio_cons > 0:
    print(f'    - En Ballotage aumentó la diferencia Izq-Der')
else:
    print(f'    - En Ballotage disminuyó la diferencia Izq-Der')

print('\n' + '='*70)

## 7. Guardar Tabla Resumen

In [ ]:
# Crear carpeta de salida
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Cleveland')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar tabla de datos
Archivo_Salida = os.path.join(Carpeta_Salida, 'Resumen_Diferencias_CT.xlsx')
df_grafico_export = df_grafico.copy()
df_grafico_export.to_excel(Archivo_Salida, index=False)

print(f'✅ Tabla resumen guardada: {Archivo_Salida}')
print(f'   {len(df_grafico_export)} ítems × {len(df_grafico_export.columns)} columnas')

## 8. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: GRÁFICOS DE CLEVELAND - DIFERENCIAS EN TIEMPO')
print('='*70)

print('\n📊 Análisis completado:')
print(f'  - Ítems analizados: {len(df_grafico)}')
print(f'  - Gráficos generados: 1 (Cleveland plot)')
print(f'  - Comparación: Generales vs Ballotage')

print('\n📁 Archivos generados:')
print('  - Cleveland_CT_Todos_Items.png (gráfico principal)')
print('  - Resumen_Diferencias_CT.xlsx (tabla de datos)')

print('\n🎯 Hallazgos clave:')
print(f'  - {aumentaron} ítems mostraron aumento en diferencia Izq-Der')
print(f'  - {disminuyeron} ítems mostraron disminución')
print(f'  - {sin_cambio} ítems sin cambio sustancial')

print('\n💡 Interpretación:')
print('  - Valores positivos: Mayor tiempo dedicado a candidatos de Izquierda')
print('  - Valores negativos: Mayor tiempo dedicado a candidatos de Derecha')
print('  - Líneas verdes: La asimetría Izq-Der aumentó en Ballotage')
print('  - Líneas naranjas: La asimetría Izq-Der disminuyó en Ballotage')

print('\n' + '='*70)
print('✅ ANÁLISIS COMPLETADO')
print('='*70)